In [1]:
from __future__ import print_function, unicode_literals, division
from io import open
import glob

In [2]:
def findFiles(path):
    return glob.glob(path)
print (findFiles('data/names/*.txt'))

[u'data/names/Polish.txt', u'data/names/Arabic.txt', u'data/names/Chinese.txt', u'data/names/German.txt', u'data/names/Dutch.txt', u'data/names/Irish.txt', u'data/names/French.txt', u'data/names/Russian.txt', u'data/names/Vietnamese.txt', u'data/names/Japanese.txt', u'data/names/Korean.txt', u'data/names/Italian.txt', u'data/names/English.txt', u'data/names/Scottish.txt', u'data/names/Greek.txt', u'data/names/Spanish.txt', u'data/names/Portuguese.txt', u'data/names/Czech.txt']


In [3]:
import unicodedata
import string

In [4]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
print (n_letters)
def unicodetoAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
print(unicodetoAscii('Ślusàrski'))

57
Slusarski


In [5]:
category_lines = {}
all_categories = []
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodetoAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
n_categories = len(all_categories)

In [6]:
print (n_categories)

18


In [7]:
print (category_lines['Italian'][:5])

[u'Abandonato', u'Abatangelo', u'Abatantuono', u'Abate', u'Abategiovanni']


In [8]:
import torch
from torch.autograd import Variable

In [9]:
def letterToIndex(letter):
    return all_letters.find(letter)
def linetoTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


In [10]:
print (linetoTensor('Jones').size())

torch.Size([5, 1, 57])


In [11]:
a = [[1, 2, 3]]
a_tensor = torch.Tensor(a)
print (a_tensor.size())
b = [1, 2, 3]
b_tensor = torch.Tensor(b)
print (b_tensor.size())

torch.Size([1, 3])
torch.Size([3])


In [12]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2o = torch.nn.Linear(input_size + hidden_size, output_size)
        self.i2h = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.softmax = torch.nn.LogSoftmax(dim=1)
        
    def forward(self, inpt, hidden):
        
        combined = torch.cat((inpt, hidden), 1)
        next_hidden = self.i2h(combined)
        out = self.i2o(combined)
        output = self.softmax(out)
        
        return output, next_hidden
    
    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))
        

In [16]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

inpt = Variable(linetoTensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))
output, next_hidden = rnn(inpt[0], hidden)
print(output.size())
print(output)

torch.Size([1, 18])
Variable containing:

Columns 0 to 9 
-2.9018 -2.9551 -2.8189 -2.9023 -2.9131 -2.8594 -2.9662 -2.9072 -2.9706 -2.8839

Columns 10 to 17 
-2.9124 -2.8030 -2.8302 -2.9448 -2.8784 -2.8184 -2.8792 -2.9037
[torch.FloatTensor of size 1x18]



In [17]:
def categoryfromOutput(output):
    top_n, top_i = output.data.topk(1)
    category_i =top_i[0][0]
    return all_categories[category_i], category_i
categoryfromOutput(output)

(u'Italian', 11)

In [18]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l)-1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(linetoTensor(line))
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print ('category =', category, '/ line =', line)
    

category = Vietnamese / line = Doan
category = Chinese / line = See 
category = Dutch / line = Maes
category = Japanese / line = Kurkawa
category = Scottish / line = Marshall
category = Portuguese / line = Moreno
category = Vietnamese / line = Vuong
category = Vietnamese / line = Sai
category = Vietnamese / line = Han
category = Dutch / line = Venn


In [26]:
criterion = torch.nn.NLLLoss()

In [28]:
learning_rate = 0.005

def train(category_tensor, line_tensor):
    
    hidden = rnn.initHidden()
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    loss = criterion(output, category_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.data[0]

In [46]:
import time
import math

n_iters = 100000
print_every = 10
plot_every = 1000
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    if iter % print_every == 0:
        guess, guess_i = categoryfromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))


    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

10 0% (0m 0s) 3.2852 Maurice / Portuguese ✗ (Irish)
20 0% (0m 0s) 0.8587 Foong / Chinese ✓
30 0% (0m 0s) 0.1230 Palmeiro / Portuguese ✓
40 0% (0m 0s) 0.3538 Gajos / Polish ✓
50 0% (0m 0s) 0.4640 Bonnaire / French ✓
60 0% (0m 0s) 1.0128 Ma / Korean ✗ (Vietnamese)
70 0% (0m 0s) 0.0802 Wojda / Polish ✓
80 0% (0m 0s) 0.1510 Esteves / Portuguese ✓
90 0% (0m 0s) 0.3445 Mclean / Scottish ✓
100 0% (0m 0s) 1.2007 Garratt / English ✓
110 0% (0m 0s) 0.8086 Liu / Vietnamese ✗ (Chinese)
120 0% (0m 0s) 2.8767 Gosse / Scottish ✗ (French)
130 0% (0m 0s) 0.0080 Timerbulatov / Russian ✓
140 0% (0m 0s) 0.5779 Mao / Chinese ✓
150 0% (0m 0s) 0.0088 Vassilopulos / Greek ✓
160 0% (0m 0s) 1.0576 Engel / German ✓
170 0% (0m 0s) 0.0486 Zielinski / Polish ✓
180 0% (0m 0s) 0.8277 Hakimi / Japanese ✗ (Arabic)
190 0% (0m 0s) 0.1169 De la fontaine / French ✓
200 0% (0m 0s) 0.0154 Szczepanski / Polish ✓
210 0% (0m 0s) 1.0174 Saigo / Japanese ✓
220 0% (0m 0s) 2.1147 Ansell / Dutch ✗ (English)
230 0% (0m 0s) 1.8541 Hol

1860 1% (0m 2s) 3.6159 Cerny / Scottish ✗ (Czech)
1870 1% (0m 2s) 0.3242 Chin / Korean ✓
1880 1% (0m 2s) 0.3453 Said / Arabic ✓
1890 1% (0m 2s) 0.4752 Mao / Chinese ✓
1900 1% (0m 2s) 0.5306 Banh / Vietnamese ✓
1910 1% (0m 2s) 0.2583 Klossner / German ✓
1920 1% (0m 2s) 0.2016 Thach / Vietnamese ✓
1930 1% (0m 2s) 1.1616 Knepp / Dutch ✗ (German)
1940 1% (0m 2s) 2.0841 Denend / French ✗ (Dutch)
1950 1% (0m 2s) 1.2390 Nunez / German ✗ (Spanish)
1960 1% (0m 2s) 2.0034 Sullivan / Irish ✗ (English)
1970 1% (0m 2s) 0.4185 Bach / Vietnamese ✓
1980 1% (0m 2s) 2.4846 Schmeiser / German ✗ (Czech)
1990 1% (0m 2s) 0.1662 Ambrosi / Italian ✓
2000 2% (0m 2s) 1.2914 Zaruba / Czech ✓
2010 2% (0m 2s) 0.4541 Hoang / Vietnamese ✓
2020 2% (0m 2s) 2.8043 Asker / German ✗ (Arabic)
2030 2% (0m 2s) 3.9398 Albero / Spanish ✗ (Italian)
2040 2% (0m 2s) 0.3790 Piatek / Polish ✓
2050 2% (0m 2s) 1.2544 Nunes / Portuguese ✓
2060 2% (0m 2s) 0.9165 Yun / Chinese ✗ (Korean)
2070 2% (0m 2s) 0.7863 Hwang / Korean ✓
2080 2% 

3680 3% (0m 5s) 0.4598 Metzger / German ✓
3690 3% (0m 5s) 1.4406 Ebina / Italian ✗ (Japanese)
3700 3% (0m 5s) 5.4228 Riagain / French ✗ (Irish)
3710 3% (0m 5s) 3.6743 Oorschot / Spanish ✗ (Dutch)
3720 3% (0m 5s) 0.3875 Badyashin / Russian ✓
3730 3% (0m 5s) 2.7999 Antopolsky / Polish ✗ (Russian)
3740 3% (0m 5s) 0.2565 Ming / Chinese ✓
3750 3% (0m 5s) 0.2161 Youj / Korean ✓
3760 3% (0m 5s) 1.5648 Bishara / Japanese ✗ (Arabic)
3770 3% (0m 5s) 0.3817 Souza / Portuguese ✓
3780 3% (0m 5s) 0.2219 Messmann / German ✓
3790 3% (0m 5s) 2.9397 Jija / Czech ✗ (Russian)
3800 3% (0m 5s) 0.0490 Trumbauer / German ✓
3810 3% (0m 5s) 1.4053 Dempko / Russian ✗ (Czech)
3820 3% (0m 5s) 1.1052 Mihaleiko / Japanese ✗ (Russian)
3830 3% (0m 5s) 1.1912 Sugimura / Japanese ✓
3840 3% (0m 5s) 0.2742 Lai / Chinese ✓
3850 3% (0m 5s) 1.4502 Ha / Vietnamese ✗ (Korean)
3860 3% (0m 5s) 0.7392 Lam / Chinese ✓
3870 3% (0m 5s) 5.1656 Bartosz / German ✗ (Polish)
3880 3% (0m 5s) 1.9849 Cummins / French ✗ (English)
3890 3% (0m

5470 5% (0m 7s) 1.5141 Kendell / Dutch ✗ (English)
5480 5% (0m 7s) 1.6696 Lachapelle / Irish ✗ (French)
5490 5% (0m 7s) 0.2288 Ming / Chinese ✓
5500 5% (0m 7s) 0.6108 Simecek / Czech ✓
5510 5% (0m 7s) 0.0013 Papadelias / Greek ✓
5520 5% (0m 7s) 1.8192 Juan / Korean ✗ (Chinese)
5530 5% (0m 7s) 0.0969 Chikurov / Russian ✓
5540 5% (0m 7s) 1.0757 Houtum / Dutch ✓
5550 5% (0m 7s) 1.0213 Gronchi / Italian ✓
5560 5% (0m 7s) 0.7887 Yang / Chinese ✗ (Korean)
5570 5% (0m 7s) 0.0067 Frangopoulos / Greek ✓
5580 5% (0m 7s) 0.0844 Donndubhan / Irish ✓
5590 5% (0m 7s) 0.2103 Hishida / Japanese ✓
5600 5% (0m 7s) 1.6615 Maier / French ✗ (German)
5610 5% (0m 7s) 0.0927 Yu / Korean ✓
5620 5% (0m 7s) 0.5307 Mo / Korean ✓
5630 5% (0m 7s) 1.4553 Rocha / Czech ✗ (Spanish)
5640 5% (0m 7s) 1.3381 Danas / Arabic ✗ (Greek)
5650 5% (0m 7s) 0.4947 Aihara / Japanese ✓
5660 5% (0m 7s) 1.6430 Allard / English ✗ (French)
5670 5% (0m 7s) 0.7698 Jez / Polish ✓
5680 5% (0m 7s) 0.8423 Rim / Korean ✓
5690 5% (0m 7s) 0.5259

7380 7% (0m 10s) 1.0719 Cham / Arabic ✓
7390 7% (0m 10s) 0.6088 Weeber / German ✓
7400 7% (0m 10s) 2.1110 Mathieu / Japanese ✗ (French)
7410 7% (0m 10s) 2.2156 Sokal / Czech ✗ (Polish)
7420 7% (0m 10s) 0.2274 De palma / Italian ✓
7430 7% (0m 10s) 0.7235 Ragno / Italian ✓
7440 7% (0m 10s) 0.1781 Reinders / Dutch ✓
7450 7% (0m 10s) 2.1928 Denis / Greek ✗ (French)
7460 7% (0m 10s) 2.0323 Fewtrell / German ✗ (English)
7470 7% (0m 10s) 2.8088 Eadie / French ✗ (English)
7480 7% (0m 10s) 2.7690 Eastaugh / Arabic ✗ (English)
7490 7% (0m 10s) 0.6486 Holzer / German ✓
7500 7% (0m 10s) 0.3546 Koury / Arabic ✓
7510 7% (0m 10s) 0.0080 Toyama / Japanese ✓
7520 7% (0m 10s) 1.7365 Tsucgimoto / Scottish ✗ (Japanese)
7530 7% (0m 10s) 0.6439 Lim  / Korean ✓
7540 7% (0m 10s) 0.6797 Fonseca / Spanish ✓
7550 7% (0m 10s) 0.5246 Gajos / Polish ✓
7560 7% (0m 10s) 1.7668 Shirko / Japanese ✗ (Russian)
7570 7% (0m 10s) 0.3669 De santigo / Portuguese ✓
7580 7% (0m 10s) 0.2187 Valdez / Spanish ✓
7590 7% (0m 10s) 0.

...

92840 92% (2m 9s) 3.1703 Kasai / Arabic ✗ (Japanese)
92850 92% (2m 9s) 2.5092 Duarte / Scottish ✗ (Portuguese)
92860 92% (2m 9s) 0.2362 Macshuibhne / Irish ✓
92870 92% (2m 9s) 0.7021 Beltz / German ✓
92880 92% (2m 9s) 1.7459 Nemecek / Polish ✗ (Czech)
92890 92% (2m 9s) 1.4423 Leeuwenhoeck / Czech ✗ (Dutch)
92900 92% (2m 9s) 0.0822 Zhi / Chinese ✓
92910 92% (2m 9s) 1.5818 Zajicek / Polish ✗ (Czech)
92920 92% (2m 9s) 2.1017 Mohren / Irish ✗ (Dutch)
92930 92% (2m 9s) 0.3902 Sun / Korean ✓
92940 92% (2m 9s) 2.7780 Cap / Vietnamese ✗ (Czech)
92950 92% (2m 9s) 0.0064 Antonopoulos / Greek ✓
92960 92% (2m 9s) 0.3445 Kourempes / Greek ✓
92970 92% (2m 9s) 1.0338 Chu / Chinese ✗ (Vietnamese)
92980 92% (2m 9s) 0.2409 Tsogas / Greek ✓
92990 92% (2m 9s) 3.1575 Santillian / Irish ✗ (Spanish)
93000 93% (2m 9s) 0.3266 Tarantino / Italian ✓
93010 93% (2m 9s) 0.0276 Vuu / Vietnamese ✓
93020 93% (2m 9s) 2.2406 Adam / Arabic ✗ (Irish)
93030 93% (2m 9s) 3.2341 Kriz / Polish ✗ (Czech)
93040 93% (2m 9s) 2.819

94610 94% (2m 11s) 3.0722 Petit / French ✗ (Spanish)
94620 94% (2m 11s) 6.7736 Murtas / Scottish ✗ (Italian)
94630 94% (2m 11s) 0.8315 Stenger / German ✓
94640 94% (2m 11s) 0.9636 Righi / Italian ✓
94650 94% (2m 11s) 0.1592 Mendes / Portuguese ✓
94660 94% (2m 11s) 1.7388 Klineberg / German ✗ (Czech)
94670 94% (2m 11s) 1.9996 Dahl / Irish ✗ (German)
94680 94% (2m 11s) 0.9817 Zambrano / Spanish ✗ (Italian)
94690 94% (2m 11s) 3.9084 Muchalon / Irish ✗ (Czech)
94700 94% (2m 11s) 0.0761 Vazquez / Spanish ✓
94710 94% (2m 11s) 1.3299 Matos / Greek ✗ (Portuguese)
94720 94% (2m 11s) 0.7012 Giang / Vietnamese ✓
94730 94% (2m 11s) 0.5145 Honjas / Greek ✓
94740 94% (2m 11s) 0.5828 Cabral / Portuguese ✓
94750 94% (2m 11s) 0.0164 O'Donnell / Irish ✓
94760 94% (2m 11s) 0.1099 Kamachi / Japanese ✓
94770 94% (2m 11s) 0.0616 Fakhoury / Arabic ✓
94780 94% (2m 11s) 0.8378 Brisbois / French ✓
94790 94% (2m 11s) 0.3993 Chu / Vietnamese ✓
94800 94% (2m 11s) 0.7684 De santigo / Portuguese ✓
94810 94% (2m 11s)

96410 96% (2m 13s) 0.0451 Rooiakkers / Dutch ✓
96420 96% (2m 13s) 1.1254 O'Hanlon / English ✗ (Irish)
96430 96% (2m 13s) 1.1733 Holman / English ✓
96440 96% (2m 13s) 0.2227 Kavanagh / Irish ✓
96450 96% (2m 13s) 1.6812 Victor / English ✗ (Czech)
96460 96% (2m 13s) 0.2780 Pak / Korean ✓
96470 96% (2m 14s) 0.9567 Sargent / French ✓
96480 96% (2m 14s) 0.1671 OwYang / Chinese ✓
96490 96% (2m 14s) 1.4399 Sam / Korean ✗ (Chinese)
96500 96% (2m 14s) 0.6448 Sitz / German ✓
96510 96% (2m 14s) 1.0331 Ngo / Korean ✗ (Vietnamese)
96520 96% (2m 14s) 0.0426 Yakunin / Russian ✓
96530 96% (2m 14s) 0.1819 Eng / Chinese ✓
96540 96% (2m 14s) 1.7983 Rompaye / English ✗ (Dutch)
96550 96% (2m 14s) 0.0414 D'antonio / Italian ✓
96560 96% (2m 14s) 0.0455 Macghabhann / Irish ✓
96570 96% (2m 14s) 1.0935 Shan / Chinese ✓
96580 96% (2m 14s) 0.3867 Walther / German ✓
96590 96% (2m 14s) 0.0507 Treasach / Irish ✓
96600 96% (2m 14s) 0.1869 Thean / Chinese ✓
96610 96% (2m 14s) 0.9396 Sitko / Polish ✓
96620 96% (2m 14s) 

98170 98% (2m 16s) 0.3709 Akatsuka / Japanese ✓
98180 98% (2m 16s) 0.7997 Delgado / Portuguese ✓
98190 98% (2m 16s) 0.3564 Hasekura / Japanese ✓
98200 98% (2m 16s) 0.2262 Eoghan / Irish ✓
98210 98% (2m 16s) 2.6921 Gately / Irish ✗ (English)
98220 98% (2m 16s) 4.7317 Groe / Scottish ✗ (German)
98230 98% (2m 16s) 0.4776 Cabral / Portuguese ✓
98240 98% (2m 16s) 0.6534 Dang / Vietnamese ✓
98250 98% (2m 16s) 1.2302 Savchak / Polish ✗ (Czech)
98260 98% (2m 16s) 0.2254 Takemura / Japanese ✓
98270 98% (2m 16s) 0.0392 Tiraboschi / Italian ✓
98280 98% (2m 16s) 0.0484 Winogrodzki / Polish ✓
98290 98% (2m 16s) 0.2510 Mizuno / Japanese ✓
98300 98% (2m 16s) 0.1058 Konishi / Japanese ✓
98310 98% (2m 16s) 0.7025 Danas / Greek ✓
98320 98% (2m 16s) 0.0034 Lebedev / Russian ✓
98330 98% (2m 16s) 0.1635 Ra / Korean ✓
98340 98% (2m 16s) 1.0795 Palomer / French ✓
98350 98% (2m 16s) 0.1682 Houtem / Dutch ✓
98360 98% (2m 16s) 0.0700 Tsai  / Korean ✓
98370 98% (2m 16s) 0.3193 Pennders / Dutch ✓
98380 98% (2m 16

99990 99% (2m 18s) 2.9129 Matos / Greek ✗ (Portuguese)
100000 100% (2m 18s) 0.0009 Rooijakkers / Dutch ✓


In [33]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

In [48]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    output = evaluate(Variable(linetoTensor(input_line)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        category_index = topi[0][i]
        print('(%.2f) %s' % (value, all_categories[category_index]))
        predictions.append([value, all_categories[category_index]])

predict('Howard')
predict('Chao')
predict('Nakamoto')


> Howard
(-0.67) English
(-1.99) Arabic
(-2.07) Scottish

> Chao
(-0.78) Chinese
(-0.89) Korean
(-3.47) Vietnamese

> Nakamoto
(-0.01) Japanese
(-5.93) Italian
(-6.44) Russian
